# Extracting the Candidates from the Markdown files

The next step is to get the candidates from the Markdown files. The assumption is that PymuPDF4LLM has extracted all candidates properly in Markdown tables. With this, we will remove all non-Markdown table data and process them based on header-matching, and sort them by the order.

In [1]:
import fs from "node:fs"
import path from "node:path"

In [2]:
let data = fs.readFileSync('./OV.md', 'utf8');

In [3]:
function sortCandidate (a: string, b: string) {
  const firstNumber = Number(a.match(/^(\d+)\.?/)[1])
  const secondNumber = Number(b.match(/^(\d+)\.?/)[1])
  return firstNumber - secondNumber
}

In [4]:
function extractCandidates (data: string) {
  let list = {}
  let currentPosition = ""
  
  // Go through the file line-by line
  
  for (const line of data.split('\n')) {
    // const line = (data as string).split('\n')[91]
    const tableMatcherRegex = /\|((\\\|)|[^\r\n\|])+/g                   // Capture cells starting with the pipe character, except if the pipe character is escaped
    let row = line.match(tableMatcherRegex)
    
    if (!row || row.length === 0) continue                               // If there are no matches, this is not a table, skip the line
    
    row = row
    .map(match => match.replace(/^\|/, "").trim())                     // Remove the starting pipe character, and remove extra whitespaces
    .filter(match => match)                                            // And remove empty strings
    
    if (row[0].replaceAll('-', '') === '') continue                      // If the first cell is a header separator (multiple dashes), skip the line
    
    const isHeader = /^[^\d].*Vote for (\d+)/.exec(row[0])               // If the first cell does not start with a number, it's probably a header
    
    if (isHeader) {
      const position = row[0].split("/")[0].trim()                       // Get the position
      const voteFor = Number(isHeader[1])                                        // Count the "vote for"
      
      if (currentPosition !== position) {                                // If the position is different from the current position being worked on,
        currentPosition = position
        list[position] = {                                               // Write a blank position property
          candidates: [],
          header: row[0],
          voteFor
        }
      }
      continue                                                            // And skip the rest
    } else {                                                              // Else, append the candidates to the row.
      list[currentPosition].candidates =
      list[currentPosition].candidates.concat(row)
    }
  }
  
  // When the list is complete, sort and count the candidates
  
  for (const position of Object.keys(list)) {
    list[position].candidates = list[position].candidates.sort(sortCandidate)
    list[position].count = list[position].candidates.length
  }

  return list
}


In [5]:
let national = extractCandidates(data)
await Deno.jupyter.display(
  { 'text/json': JSON.stringify(national, null, 2) },
  { raw: true }
)


Nice. We finished the basic extraction of candidates to a list.

The partylist count does not match as the [Wage Hike partylist withdrew their candidacy](https://www.inquirer.net/423441/party-list-group-wage-hike-withdraws-bid-from-2025-polls/) last December 17, 2024.

We now write the results down into `.json` files.

In [84]:
fs.writeFileSync('./national.json', JSON.stringify(national))

Let's test the code to include local candidates and add some sanity checks with it.

In [87]:
let akbarRaw = fs.readFileSync('./md/BARMM.BASILAN.AKBAR.md', 'utf8')

In [6]:
function extractCandidatesWithSanityChecks (data: string) {
  let nationalPosts = Object.keys(national)
  let list = extractCandidates(data);

  if (nationalPosts.some(nationalPost => list[nationalPost].count !== national[nationalPost].count)) {
    throw new Error(`${nationalPost} count mismatch`)
  }
  
  for (const nationalPost of nationalPosts) {
    delete list[nationalPost]
  }

  return list
}

In [99]:
let akbar = extractCandidatesWithSanityChecks(akbarRaw)
await Deno.jupyter.display(
  { 'text/json': JSON.stringify(akbar, null, 2) },
  { raw: true }
)

Time to run it for all files.

In [9]:
const markdownFiles = fs.readdirSync('./md')

In [7]:
let prevPercent = 0.0

for (const [index, markdownFile] of markdownFiles.entries()) {
  let markdownString = fs.readFileSync(`./md/${markdownFile}`, 'utf8')
  let cityMuniName = path.parse(markdownFile).name
  let cityMuni = ""

  try {
    cityMuni = extractCandidatesWithSanityChecks(markdownString)
  } catch (e) {
    console.error("Error at: ", cityMuniName, e);
  }
  
  if (cityMuni === "") throw new Error("Failed to read ", cityMuniName)

  fs.writeFileSync(`./json/${cityMuniName}.json`, JSON.stringify(cityMuni))

  let percent = ((index + 1) / markdownFiles.length) * 100.0
  if (Math.floor(prevPercent / 10.0) < Math.floor(percent / 10.0)) {
    console.log(`${Math.round(percent)}% complete`)
    prevPercent = percent
  }
}

10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
100% complete


100

## Aggregate Stats 

With everything parsed, we now get the total number of candidates, the positions to elect them in, and if there are any discrepancies.

In [12]:
// import { display } from 'https://deno.land/x/display/mod.ts'
import * as pl from 'npm:nodejs-polars'


In [13]:
const jsonFiles = fs.readdirSync('./json')

In [10]:
let prevPercent = 0.0

let candidateTally = {}

for (const position of Object.keys(national)) {
  candidateTally[position] = {
    voteFor: national[position].voteFor || 0,
    count: national[position].count || 0,
  }
}

for (const [index, jsonFile] of jsonFiles.entries()) {
  let jsonString = fs.readFileSync(`./json/${jsonFile}`, 'utf8')
  // let cityMuniName = path.parse(jsonFile).name
  let cityMuni = JSON.parse(jsonString)

  for (const position of Object.keys(cityMuni)) {
    candidateTally[position] = {
      voteFor:
        (cityMuni[position]?.voteFor ?? 0),
      count: (candidateTally[position]?.count ?? 0) + cityMuni[position].count ?? 0,
    }
  }

  let percent = ((index + 1) / markdownFiles.length) * 100.0
  if (Math.floor(prevPercent / 10.0) < Math.floor(percent / 10.0)) {
    console.log(`${Math.round(percent)}% complete`)
    prevPercent = percent
  }
}

let dataFrameObj = {
  position: [],
  voteFor: [],
  count: [],
}

for (const position of Object.keys(candidateTally)) {
  dataFrameObj.position.push(position)
  dataFrameObj.voteFor.push(candidateTally[position].voteFor)
  dataFrameObj.count.push(candidateTally[position].count)
}

let dataFrame = new pl.DataFrame(dataFrameObj)

dataFrame
// await display(dataFrame)

10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
100% complete


position,voteFor,count
SENATOR,12,66
PARTY LIST,1,155
"MEMBER, HOUSE OF REPRESENTATIVES",1,3931
PROVINCIAL GOVERNOR,1,4571
PROVINCIAL VICE-GOVERNOR,1,3903
"MEMBER, SANGGUNIANG PANLALAWIGAN",5,12336
MAYOR,1,3951
VICE-MAYOR,1,3622
"MEMBER, SANGGUNIANG BAYAN",8,27427
BARMM PARTY REPRESENTATIVES,1,763


## Generating the local mapping file

To include candidates for local posts, we have the user search their province and city/municipality they are registered in. Sometimes, cities are further divided into districts.

From the filename of the Markdown files, we will now generate a CSV file containing the following:
- Identifier: The Markdown filename (format: `<region>.<province>.<cityMunicipality>` or `NCR.<cityMunicipality>`)
- Province: The name of the province (or NCR)
- City (and District) or Municipality: The name of the city (and district) or municipality

We then will use the Philippine Standard Geographic Code (PSGC) of 3Q 2024 ([source](https://psa.gov.ph/classification/psgc), up to date as of September 30, 2024) to align proper spelling of the provinces and cities/municipalities through Levenshtein distance.

In [21]:
let dataFrameMappingObj = {
  identifier: [],
  province: [],
  cityMunicipality: []
}

for (const markdownFile of markdownFiles) {
  let identifier = path.parse(markdownFile).name
  let splits = identifier.split('.')

  if (splits.length === 2) {
    dataFrameMappingObj.identifier.push(identifier)
    dataFrameMappingObj.province.push(splits[0])
    dataFrameMappingObj.cityMunicipality.push(splits[1])
  }

  if (splits.length === 3) {
    dataFrameMappingObj.identifier.push(identifier)
    dataFrameMappingObj.province.push(splits[1])                          // Skip the region
    dataFrameMappingObj.cityMunicipality.push(splits[2])
  }
}

1681

In [164]:
let dataFrameMapping = new pl.DataFrame(dataFrameMappingObj)
  .sort('province')
  .select(
    'identifier',
    pl.col('province').str.replaceAll('_', ' '),
    pl.col('cityMunicipality').str.replaceAll('_', ' ')
  )
  
dataFrameMapping


identifier,province,cityMunicipality
BFT_CAR.ABRA.BANGUED,ABRA,BANGUED
BFT_CAR.ABRA.BOLINEY,ABRA,BOLINEY
BFT_CAR.ABRA.BUCAY,ABRA,BUCAY
BFT_CAR.ABRA.BUCLOC,ABRA,BUCLOC
BFT_CAR.ABRA.DAGUIOMAN,ABRA,DAGUIOMAN
BFT_CAR.ABRA.DANGLAS,ABRA,DANGLAS
BFT_CAR.ABRA.DOLORES,ABRA,DOLORES
BFT_CAR.ABRA.LACUB,ABRA,LACUB
BFT_CAR.ABRA.LAGANGILANG,ABRA,LAGANGILANG
BFT_CAR.ABRA.LAGAYAN,ABRA,LAGAYAN


In [165]:
let dataFrameProvinces = dataFrameMapping
  .select(pl.col('province').unique().sort().str.replaceAll('_', ' '))
  .toSeries()
  .toArray()
  
dataFrameProvinces

[
  "ABRA",              "AGUSAN DEL NORTE",   "AGUSAN DEL SUR",
  "AKLAN",             "ALBAY",              "ANTIQUE",
  "APAYAO",            "AURORA",             "BASILAN",
  "BATAAN",            "BATANES",            "BATANGAS",
  "BENGUET",           "BILIRAN",            "BOHOL",
  "BUKIDNON",          "BULACAN",            "CAGAYAN",
  "CAMARINES NORTE",   "CAMARINES SUR",      "CAMIGUIN",
  "CAPIZ",             "CATANDUANES",        "CAVITE",
  "CEBU",              "COTABATO",           "DAVAODELNORTE",
  "DAVAODELSUR",       "DAVAODEORO",         "DAVAOOCCIDENTAL",
  "DAVAOORIENTAL",     "DINAGAT ISLANDS",    "EASTERN SAMAR",
  "GUIMARAS",          "IFUGAO",             "ILOCOS SUR",
  "ILOCOSNORTE",       "ILOILO",             "ISABELA",
  "KALINGA",           "LAGUNA",             "LANAODELSUR",
  "LAUNION",           "LDN",                "LEYTE",
  "MAGUINDANAODELSUR", "MAGUINDANAONORTE",   "MARINDUQUE",
  "MASBATE",           "MISOR",              "MISSOCC",
  "MPROVINCE

In [166]:
// Here's a list of all provinces from the PSGC, with NCR and SGA added at the end.
let PSGCprovinceswithcoding = pl.readCSV('./psgc/provinces.csv', {
  dtypes: { province: pl.DataType.String, code: pl.DataType.String },
})

let PSGCprovinces = PSGCprovinceswithcoding["province"].toArray()

In [167]:
`${PSGCprovinces.length} ${dataFrameProvinces.length}`

"84 84"

In [168]:
import { distance, closest } from 'https://deno.land/x/fastest_levenshtein/mod.ts'

In [ ]:
let provinceCorrections = {
  "LDN": "LANAO DEL NORTE",
  "MISOR": "MISAMIS ORIENTAL",
  "MISSOCC": "MISAMIS OCCIDENTAL",
  "MPROVINCE": "MOUNTAIN PROVINCE",
  "NCR": "NATIONAL CAPITAL REGION (NCR) - METRO MANILA",
  "SGA": "SPECIAL GEOGRAPHIC AREA",
}

let dataFrameBetterProvinces = new pl.DataFrame({
  old: dataFrameProvinces,
  new: dataFrameProvinces.map((province) => {
    if (provinceCorrections[province]) {
      return provinceCorrections[province]
    } else {
      return closest(
        province,
        pl.Series(PSGCprovinces).str.toUpperCase().toArray()
      )
    }
  }),
  score: dataFrameProvinces.map((province) => {
    if (provinceCorrections[province]) {
      return -1
    } else {
      return distance(province, closest(
        province,
        pl.Series(PSGCprovinces).str.toUpperCase().toArray()
      ))
    }
  }),
})

await Deno.jupyter.display({"text/csv": dataFrameBetterProvinces.sort("score", true).writeCSV().toString()}, {raw: "true"})

In [170]:
dataFrameBetterProvinces
  .select(pl.col('new').unique().sort())
  .toSeries()
  .toArray().length

84

In [171]:
// Here's a list of all cities and municipalities with their corresponding code. I have added old names to the end of the list with the same code as their new ones so that there are more chances to match.
let PSGCcitymuniwithcoding = pl.readCSV('./psgc/citiesMunicipalities.csv', {
  dtypes: { cityMunicipality: pl.DataType.String, code: pl.DataType.String },
})

In [172]:
// We aggregate the original table with the better province and the province code.
let joinBetterProvince = dataFrameMapping
  .join(
    dataFrameBetterProvinces.select(
      pl.col('old').alias('province'),
      pl.col('new').alias('newProvince')
    ),
    { on: 'province' }
  )
  .join(
    PSGCprovinceswithcoding.select(
      pl.col('province').alias('newProvince').str.toUpperCase(), 
      pl.col('code').alias('provinceCode')
    ), 
    { on: 'newProvince'}
  )

  joinBetterProvince

identifier,province,cityMunicipality,newProvince,provinceCode
BFT_CAR.ABRA.BANGUED,ABRA,BANGUED,ABRA,14001
BFT_CAR.ABRA.BOLINEY,ABRA,BOLINEY,ABRA,14001
BFT_CAR.ABRA.BUCAY,ABRA,BUCAY,ABRA,14001
BFT_CAR.ABRA.BUCLOC,ABRA,BUCLOC,ABRA,14001
BFT_CAR.ABRA.DAGUIOMAN,ABRA,DAGUIOMAN,ABRA,14001
BFT_CAR.ABRA.DANGLAS,ABRA,DANGLAS,ABRA,14001
BFT_CAR.ABRA.DOLORES,ABRA,DOLORES,ABRA,14001
BFT_CAR.ABRA.LACUB,ABRA,LACUB,ABRA,14001
BFT_CAR.ABRA.LAGANGILANG,ABRA,LAGANGILANG,ABRA,14001
BFT_CAR.ABRA.LAGAYAN,ABRA,LAGAYAN,ABRA,14001


In [177]:
// It's 3:31 AM and my code is getting unreadable. I will take a break here and resume later.
// What this does is it tries to create a new table where the cities and municipalities are matched to the PSGC list, and the probability score is computed such that the ones with the most distant scores can be fixed manually.

let cityMuniCorrections = {
  'SAMALISLAND KAPUTIAN DIST':
    'ISLAND GARDEN CITY OF SAMAL (KAPUTIAN DISTRICT)',
  'SAMALISLAND BABAK DIST': 'ISLAND GARDEN CITY OF SAMAL (BABAK DISTRICT)',
  'SAMALISLAND SAMAL DIST': 'ISLAND GARDEN CITY OF SAMAL (SAMAL DISTRICT)',
  'CITY OF PUERTO PRINCESA': 'CITY OF PUERTO PRINCESA',
  CEBUCITY1STDISTRICT: 'CITY OF CEBU (1ST DISTRICT)',
  CEBUCITY2NDDISTRICT: 'CITY OF CEBU (2ND DISTRICT)',
  'COTABATO CITY 1ST': 'CITY OF COTABATO (1ST DISTRICT)',
  'COTABATO CITY 2ND': 'CITY OF COTABATO (2ND DISTRICT)',
  'CITY OF GENERAL SANTOS': 'CITY OF GENERAL SANTOS',
  'CITY OF OLONGAPO': 'CITY OF OLONGAPO',
  'DAVAOCITY 1ST DIST': 'CITY OF DAVAO (1ST DISTRICT)',
  'DAVAOCITY 2ND DIST': 'CITY OF DAVAO (2ND DISTRICT)',
  'DAVAOCITY 3RD DIST': 'CITY OF DAVAO (3RD DISTRICT)',
  'CITY OF SORSOGON BACON DIST': 'CITY OF SORSOGON (BACON DISTRICT)',
  'CITY OF SORSOGON EAST DIST': 'CITY OF SORSOGON (EAST DISTRICT)',
  'CITY OF SORSOGON WEST DIST': 'CITY OF SORSOGON (WEST DISTRICT)',
  'ZAMBOANGACITY 1ST': 'CITY OF ZAMBOANGA (1ST DISTRICT)',
  'ZAMBOANGACITY 2ND': 'CITY OF ZAMBOANGA (2ND DISTRICT)',
  'ISABELA CITY': 'CITY OF ISABELA',
  MALAYBALAYCITY: 'CITY OF MALAYBALAY',
  'CITY OF MABALACAT': 'CITY OF MABALACAT',
  'TAGBILARAN CITY': 'CITY OF TAGBILARAN',
  VALENCIACITY: 'CITY OF VALENCIA',
  'CITY OF BACOOR 1ST DIST': 'CITY OF BACOOR (1ST DISTRICT)',
  'CITY OF BACOOR 2ND DIST': 'CITY OF BACOOR (2ND DISTRICT)',
  'CDOCITY 1ST': 'CITY OF CAGAYAN DE ORO (1ST DISTRICT)',
  'CDOCITY 2ND': 'CITY OF CAGAYAN DE ORO (2ND DISTRICT)',
  ELSALVADORCITY: 'CITY OF EL SALVADOR',
  'CITY OF CALOOCAN 1ST DIST': 'CITY OF CALOOCAN (1ST DISTRICT)',
  'CITY OF CALOOCAN 2ND DIST': 'CITY OF CALOOCAN (2ND DISTRICT)',
  'CITY OF CALOOCAN 3RD DIST': 'CITY OF CALOOCAN (3RD DISTRICT)',
  'CITY OF LAS PIÑAS 1ST DIST': 'CITY OF LAS PIÑAS (1ST DISTRICT)',
  'CITY OF LAS PIÑAS 2ND DIST': 'CITY OF LAS PIÑAS (2ND DISTRICT)',
  'CITY OF MAKATI 1ST DIST': 'CITY OF MAKATI (1ST DISTRICT)',
  'CITY OF MAKATI 2ND DIST': 'CITY OF MAKATI (2ND DISTRICT)',
  'CITY OF MALABON 1ST DIST': 'CITY OF MALABON (1ST DISTRICT)',
  'CITY OF MALABON 2ND DIST': 'CITY OF MALABON (2ND DISTRICT)',
  'CITY OF MANDALUYONG 1ST DIST': 'CITY OF MANDALUYONG (1ST DISTRICT)',
  'CITY OF MANDALUYONG 2ND DIST': 'CITY OF MANDALUYONG (2ND DISTRICT)',
  'CITY OF MANILA 1ST DIST': 'CITY OF MANILA (1ST DISTRICT)',
  'CITY OF MANILA 2ND DIST': 'CITY OF MANILA (2ND DISTRICT)',
  'CITY OF MANILA 3RD DIST': 'CITY OF MANILA (3RD DISTRICT)',
  'CITY OF MANILA 4TH DIST': 'CITY OF MANILA (4TH DISTRICT)',
  'CITY OF MANILA 5TH DIST': 'CITY OF MANILA (5TH DISTRICT)',
  'CITY OF MANILA 6TH DIST': 'CITY OF MANILA (6TH DISTRICT)',
  'CITY OF MARIKINA 1ST DIST': 'CITY OF MARIKINA (1ST DISTRICT)',
  'CITY OF MARIKINA 2ND DIST': 'CITY OF MARIKINA (2ND DISTRICT)',
  'CITY OF MUNTINLUPA 1ST DIST': 'CITY OF MUNTINLUPA (1ST DISTRICT)',
  'CITY OF MUNTINLUPA 2ND DIST': 'CITY OF MUNTINLUPA (2ND DISTRICT)',
  'CITY OF NAVOTAS 1ST DIST': 'CITY OF NAVOTAS (1ST DISTRICT)',
  'CITY OF NAVOTAS 2ND DIST': 'CITY OF NAVOTAS (2ND DISTRICT)',
  'CITY OF PARAÑAQUE 1ST DIST': 'CITY OF PARAÑAQUE (1ST DISTRICT)',
  'CITY OF PARAÑAQUE 2ND DIST': 'CITY OF PARAÑAQUE (2ND DISTRICT)',
  'CITY OF PASIG 1ST DIST': 'CITY OF PASIG (1ST DISTRICT)',
  'CITY OF PASIG 2ND DIST': 'CITY OF PASIG (2ND DISTRICT)',
  'CITY OF SAN JUAN 1ST DIST': 'CITY OF SAN JUAN (1ST DISTRICT)',
  'CITY OF SAN JUAN 2ND DIST': 'CITY OF SAN JUAN (2ND DISTRICT)',
  'CITY OF TAGUIG 1ST DIST': 'CITY OF TAGUIG (1ST DISTRICT)',
  'CITY OF TAGUIG 2ND DIST': 'CITY OF TAGUIG (2ND DISTRICT)',
  'CITY OF VALENZUELA 1ST DIST': 'CITY OF VALENZUELA (1ST DISTRICT)',
  'CITY OF VALENZUELA 2ND DIST': 'CITY OF VALENZUELA (2ND DISTRICT)',
  'CITY OF PASAY 1ST DIST': 'CITY OF PASAY (1ST DISTRICT)',
  'CITY OF PASAY 2ND DIST': 'CITY OF PASAY (2ND DISTRICT)',
  'PATEROS 1ST DIST': 'PATEROS (1ST DISTRICT)',
  'PATEROS 2ND DIST': 'PATEROS (2ND DISTRICT)',
  'QUEZON CITY 1ST DIST': 'QUEZON CITY (1ST DISTRICT)',
  'QUEZON CITY 2ND DIST': 'QUEZON CITY (2ND DISTRICT)',
  'QUEZON CITY 3RD DIST': 'QUEZON CITY (3RD DISTRICT)',
  'QUEZON CITY 4TH DIST': 'QUEZON CITY (4TH DISTRICT)',
  'QUEZON CITY 5TH DIST': 'QUEZON CITY (5TH DISTRICT)',
  'QUEZON CITY 6TH DIST': 'QUEZON CITY (6TH DISTRICT)',
}

joinBetterProvince
  .select('newProvince', 'provinceCode', 'cityMunicipality')
  .join(
    new pl.DataFrame({
      cityMunicipality: joinBetterProvince['cityMunicipality'],
      newCityMunicipality: joinBetterProvince['cityMunicipality']
        .toArray()
        .map((cityMuni: string, index: number) => {
          if (cityMuniCorrections[cityMuni]) {
            return cityMuniCorrections[cityMuni]
          } else {
            return closest(
              cityMuni,
              PSGCcitymuniwithcoding.filter(
                pl
                  .col('code')
                  .str.contains(
                    RegExp(`^${joinBetterProvince['provinceCode'].get(index)}`)
                  )
              )
                ['cityMunicipality'].str.toUpperCase()
                .toArray()
            )
          }
          
        }),
      score: joinBetterProvince['cityMunicipality']
        .toArray()
        .map((cityMuni: string, index: number) => {
          if (cityMuniCorrections[cityMuni]) {
            return -1
          } else {            
            return distance(
              cityMuni,
              closest(
                cityMuni,
                PSGCcitymuniwithcoding.filter(
                  pl
                  .col('code')
                  .str.contains(
                    RegExp(`^${joinBetterProvince['provinceCode'].get(index)}`)
                  )
                )
                ['cityMunicipality'].str.toUpperCase()
                .toArray()
              )
            )
          }
        }),
    }),
    { on: 'cityMunicipality' }
  )
  .sort('score', true)


newProvince,provinceCode,cityMunicipality,newCityMunicipality,score
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF PASIG 1ST DIST,CITY OF PASIG,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF PASIG 2ND DIST,CITY OF PASIG,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF SAN JUAN 1ST DIST,CITY OF SAN JUAN,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF SAN JUAN 2ND DIST,CITY OF SAN JUAN,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF TAGUIG 1ST DIST,CITY OF TAGUIG,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF TAGUIG 2ND DIST,CITY OF TAGUIG,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF VALENZUELA 1ST DIST,CITY OF VALENZUELA,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,CITY OF VALENZUELA 2ND DIST,CITY OF VALENZUELA,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,PASAY CITY 1ST DIST,PASAY CITY,9
NATIONAL CAPITAL REGION (NCR) - METRO MANILA,138,PASAY CITY 2ND DIST,PASAY CITY,9


: 

In [130]:
// PSGCcitymuniwithcoding['cityMunicipality'].str.toUpperCase().toArray()

// PSGCcitymuniwithcoding.filter(pl.col('code').str.contains(RegExp(`^${14001}`)))['cityMunicipality'].str.toUpperCase().toArray()

PSGCcitymuniwithcoding['code'].get(0)


"1731500000"